In [50]:
import pandas as pd
import glob
import os
import re
from pandas import ExcelWriter
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
#from fuzzy_expert.variable import FuzzyVariable
os.chdir("C:/Users/Mpho Bohloko/Desktop/science/DataBR")
df = pd.read_excel("C:/Users/Mpho Bohloko/Desktop/science/DataBR/Sample.xlsx",sheet_name="3. Worklist done by researchers", skiprows=1, engine='openpyxl')
#df1 = pd.read_excel("C:/Users/Mpho Bohloko/Desktop/science/DataBR/Sample.xlsx",sheet_name="1. Original list received from ", engine='openpyxl')
df.head()


,eC ID,Account ID,Account Name,Clean Website,Contact ID,First Name,Last Name,Title,New Title,Job Level,...,Lead Fixed,QA Status,QA,QA Date,Reason for Needs Fixing / Invalid,QA Notes,Direct Phone,Mobile Phone,Phone Checked?,Phone Researcher
0,eC_dhime_001,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,hospitalsaocamilosp.org.br,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,New Business and Market Intelligence Consultant,Staff Level,...,NaN,Valid,Undisclosed info,11/24/2022,NaN,NaN,NaN,NaN,Yes,Undisclosed info
1,eC_dhime_003,0013y00001Y0QYW,"Typhoon Sports Coalition, S.A.P.I. de C.V.",typhoonmx.com,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,Operations Manager,Manager Level,...,NaN,Valid,Undisclosed info,11/24/2022,NaN,NaN,NaN,NaN,Yes,Undisclosed info
2,eC_dhime_004,0013y00001Y0Vwz,Mobalpa,mobalpa.fr,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,International Operational Marketing Project Ma...,Manager Level,...,NaN,Valid,Undisclosed info,11/24/2022,NaN,NaN,NaN,NaN,Yes,Undisclosed info
3,eC_dhime_005,0013y00001Y0PEO,Ocea,ocea.fr,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,NaN,Director Level,...,NaN,Valid,Undisclosed info,11/24/2022,NaN,NaN,NaN,33 6 08 65 86 47,Yes,Undisclosed info
4,eC_dhime_006,0013y00001cykLS,National Emergencies Trust,nationalemergenciestrust.org.uk,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,NaN,Director Level,...,NaN,Valid,Undisclosed info,11/24/2022,NaN,NaN,NaN,NaN,Yes,Undisclosed info


In [51]:
#checking missing data(Check Yes rows without Website,Check Yes rows without Email status)
df.isna().sum()

eC ID                                 0
Account ID                            0
Account Name                          0
Clean Website                         0
Contact ID                            0
First Name                            0
Last Name                             0
Title                                 0
New Title                            63
Job Level                            15
Job Function                         15
email                                 0
Email Domain                          0
ZB Status                             0
New Email                            92
Email Verification                   12
Email Status                          5
Contact LinkedIn URL                  7
Other Title Verification             93
Twitter URL                          93
Contact Street                       19
Contact City                         19
Contact State                        54
Contact Zip Code                     19
Contact Country                      19


In [53]:
#checking consitency of Titlechecking uniqueness of answers
for col in df[['Title','Job Level','Job Function']]:
    df[col] = df[col].str.strip()
    print('Number of unique values in ' + str(col) +': ' + str(df[col].nunique()))

Number of unique values in Title: 97
Number of unique values in Job Level: 5
Number of unique values in Job Function: 12


In [54]:
# Auto Generate Fuzzy matching by comparing First/Last Names Matching with emails  valid 80> and 90> above match all
df['LastName_Email'] = df.apply(lambda x: fuzz.partial_ratio(x['Last Name'], x['email']), axis=1)
df['FirstName_Email'] = df.apply(lambda x: fuzz.partial_ratio(x['First Name'], x['email']), axis=1)
#match emails with website 70% and above correct
df['Website_Email'] = df.apply(lambda x: fuzz.partial_ratio(x['email'], x['Clean Website']), axis=1)
# automate creating valid and invalid by matching email with website
df['MatchingEmail'] = np.where((df.Website_Email > 70) & (df.LastName_Email > 0) | (df.FirstName_Email > 0), 'Valid', 'Invalid')
df['MatchingFirst'] = np.where((df.FirstName_Email > 70) | (df.LastName_Email > 0), 'Valid', 'Mismatch between First/Last Name and Email')
df['MatchingLast'] = np.where((df.FirstName_Email > 0) | (df.LastName_Email > 70), 'Valid', 'Mismatch between First/Last Name and Email')
df.head(10)

,eC ID,Account ID,Account Name,Clean Website,Contact ID,First Name,Last Name,Title,New Title,Job Level,...,Direct Phone,Mobile Phone,Phone Checked?,Phone Researcher,LastName_Email,FirstName_Email,Website_Email,MatchingEmail,MatchingFirst,MatchingLast
0,eC_dhime_001,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,hospitalsaocamilosp.org.br,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,New Business and Market Intelligence Consultant,Staff Level,...,NaN,NaN,Yes,Undisclosed info,80,29,42,Valid,Valid,Valid
1,eC_dhime_003,0013y00001Y0QYW,"Typhoon Sports Coalition, S.A.P.I. de C.V.",typhoonmx.com,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,Operations Manager,Manager Level,...,NaN,NaN,Yes,Undisclosed info,83,86,23,Valid,Valid,Valid
2,eC_dhime_004,0013y00001Y0Vwz,Mobalpa,mobalpa.fr,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,International Operational Marketing Project Ma...,Manager Level,...,NaN,NaN,Yes,Undisclosed info,83,29,80,Valid,Valid,Valid
3,eC_dhime_005,0013y00001Y0PEO,Ocea,ocea.fr,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,NaN,Director Level,...,NaN,33 6 08 65 86 47,Yes,Undisclosed info,91,50,100,Valid,Valid,Valid
4,eC_dhime_006,0013y00001cykLS,National Emergencies Trust,nationalemergenciestrust.org.uk,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,NaN,Director Level,...,NaN,NaN,Yes,Undisclosed info,88,75,100,Valid,Valid,Valid
5,eC_dhime_007,0013y00001cfjmN,Funda??o Chesf de Assist?ncia e Seguridade Social,fachesf.com.br,0033y00002ON1HvAAL,Mario,Carvalho,IT Manager,NaN,NaN,...,NaN,55 81 98229-5489,Yes,Undisclosed info,38,60,57,Valid,Valid,Valid
6,eC_dhime_009,0013y00001Y0VuC,Fondation A.R.H.M,arhm.fr,0033y00002OGFUmAAP,Agnes,Marie-Egyptienne,MD/Chief Operating Officer (Directeur General),Managing Director,C Level,...,NaN,NaN,Yes,Undisclosed info,88,80,100,Valid,Valid,Valid
7,eC_dhime_010,0013y00001Y0Sra,Oilers Entertainment Group,nhl.com,0033y00002OwopyAAB,Abe,Hajar,VP Corporate Partnerships,NaN,Vice President Level,...,NaN,1 780-984-1170,Yes,Undisclosed info,80,33,57,Valid,Valid,Valid
8,eC_dhime_011,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",summitmx.com,0033y00002P3blxAAB,Ai,Ogihara,Director,NaN,Director Level,...,NaN,NaN,Yes,Undisclosed info,86,50,100,Valid,Valid,Valid
9,eC_dhime_012,0013y00001Y0Qbu,"Servilamina Summit Mexicana, S.A. de C.V.",summitmx.com,0033y00002P3blwAAB,Aida,Mendoza,Gerente General,General Manager,C Level,...,NaN,NaN,Yes,Undisclosed info,29,75,100,Valid,Valid,Valid


In [55]:
#Renaming variables to be similar with delivery tabs and droping unwanted variables
df.columns
df = df.rename(columns = {'Contact LinkedIn URL': 'LinkedIn URL','Contact Street':'Mailing Street','Contact City':'Mailing City',
                         'Contact State':'Mailing State/Province','Contact Zip Code':'Mailing Zip/Postal Code',
                          'Contact Country':'Mailing Country','Office Location Phone':'Mobile Phone'})
df = df[['eC ID','LinkedIn URL','New Title', 'Job Level',
       'Job Function', 'New Email', 'Email Status','Mailing Street', 'Mailing City','Mailing State/Province',
         'Mailing Zip/Postal Code','Mailing Country','Mobile Phone','Still at the company?',
       'MatchingEmail', 'MatchingFirst','MatchingLast']]
df

,eC ID,LinkedIn URL,New Title,Job Level,Job Function,New Email,Email Status,Mailing Street,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast
0,eC_dhime_001,https://www.linkedin.com/in/adriano-alves-4140...,New Business and Market Intelligence Consultant,Staff Level,Sales,NaN,"Given email private, no new corporate email added","Rua Voluntários da Pátria, 3693, Santana",Sao Paulo,NaN,02401-300,Brazil,55 11 2972 8000,NaN,Yes,Valid,Valid,Valid
1,eC_dhime_003,https://www.linkedin.com/in/abraham-vargas-937...,Operations Manager,Manager Level,Operations,NaN,"Given email for older employment, no new corpo...","Cerrada de Loma Bonita 33, Miguel Hidalgo",Mexico City,NaN,11950,Mexico,NaN,NaN,Yes,Valid,Valid,Valid
2,eC_dhime_004,https://www.linkedin.com/in/auriane-breton,International Operational Marketing Project Ma...,Manager Level,Marketing,NaN,Given email verified,18 Rue des Vernaies,Thones,NaN,74230,France,33 4 88 78 89 09,NaN,Yes,Valid,Valid,Valid
3,eC_dhime_005,https://www.linkedin.com/in/amaury-de-cacquera...,NaN,Director Level,Sales,NaN,Given email verified,Quai de la Cabaude,Les Sables d'Olonne,NaN,85100,France,33 2 51 21 05 90,33 6 08 65 86 47,Yes,Valid,Valid,Valid
4,eC_dhime_006,https://www.linkedin.com/in/adil-husseini-0b999ba,NaN,Director Level,Marketing,NaN,Given email verified,"London Scottish House, 95 Horseferry Road",London,England,SW1P 2DX,United Kingdom,NaN,NaN,Yes,Valid,Valid,Valid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eC_dhime_334,NaN,NaN,NaN,NaN,NaN,Given email verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid
96,eC_dhime_372,https://www.linkedin.com/in/sara-dramé-b14867150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Valid,Valid,Valid
97,eC_dhime_386,https://www.linkedin.com/in/sébastien-robert-9...,NaN,NaN,NaN,NaN,Given email verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Valid,Valid,Valid
98,eC_dhime_402,https://www.linkedin.com/in/tristan-goodes,NaN,NaN,NaN,NaN,Given email verified,NaN,NaN,NaN,NaN,NaN,NaN,1 778-233-1334,No,Valid,Valid,Valid


In [56]:
# verify original dataset with dataset done by reseachers on similar variables
df1 = pd.read_excel("C:/Users/Mpho Bohloko/Desktop/science/DataBR/Sample.xlsx",sheet_name="1. Original list received from ", engine='openpyxl')
df1 = df1.rename(columns = {'eC_ID': 'eC ID'})
#Merge the two datasets using EC ID  to check quality
dft = pd.merge(df1, df, on="eC ID", how="outer")
dft

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,Sao Paulo,NaN,02401-300,Brazil,55 11 2972 8000,NaN,Yes,Valid,Valid,Valid
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,Mexico City,NaN,11950,Mexico,NaN,NaN,Yes,Valid,Valid,Valid
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,Thones,NaN,74230,France,33 4 88 78 89 09,NaN,Yes,Valid,Valid,Valid
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,Les Sables d'Olonne,NaN,85100,France,33 2 51 21 05 90,33 6 08 65 86 47,Yes,Valid,Valid,Valid
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,London,England,SW1P 2DX,United Kingdom,NaN,NaN,Yes,Valid,Valid,Valid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid
96,eC_dhime_372,0033y00002JaQdqAAF,sara,Dramé,Custumer Care Manager,sara@anybuddyapp.com,EURATECHNOLOGIES,Lille,FR-59,59000,...,NaN,NaN,NaN,NaN,NaN,NaN,No,Valid,Valid,Valid
97,eC_dhime_386,0033y00002PNFcIAAX,Sébastien,Robert,Directeur du service de relations de travail,srobert@aptsq.com,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,No,Valid,Valid,Valid
98,eC_dhime_402,0033y00002PQGxpAAH,Tristan,Goodes,3D Asset Supervisor,tgoodes@atomiccartoons.com,123 7th Ave W,Vancouver,BC,V5Y 1L8,...,NaN,NaN,NaN,NaN,NaN,1 778-233-1334,No,Valid,Valid,Valid


In [71]:
#LinkedIn Duplicates Check 7 clients duplicated
duplinked = dft[dft['LinkedIn URL'].duplicated()]
duplinked

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast
44,eC_dhime_051,0033y00002N3CGWAA3,Brent,Lewis,"Executive Vice President, Digital",brent_lewis@harlequin.ca,22 Adelaide Street W 41St Fl,Toronto,ON,M5H 4E3,...,Toronto,Ontario,M5H 4E3,Canada,(888) 432-4879,NaN,Yes - Duplicate,Valid,Valid,Valid
68,eC_dhime_077,0033y00002PYVoQAAX,Clare,Chislett,Head of Operations,clare@aidboxcommunity.co.uk,NaN,NaN,NaN,NaN,...,Bristol,England,BS6 5RE,United Kingdom,44 117 336 8441,NaN,Yes,Valid,Valid,Valid
86,eC_dhime_164,0033y00002O1fs9AAB,Gary,Smith,Fire Chief,gsmith@gomontrose.com,3960 Medina Rd,Fairlawn,OH,44333,...,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid
87,eC_dhime_167,0033y00002OGyC6AAL,Gustavo,Lima Olmos Ferraz,Auxiliar de escritório,guolmos06@gmail.com,NaN,NaN,SP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid
91,eC_dhime_271,0033y00002OkX3eAAF,Madeline,Rutherford,Development Officer,madeline.rutherfoord@inspirenorth.co.uk,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid
93,eC_dhime_290,0033y00002NR8d4AAD,Michaël,Ruijter,Développeur et chargé de projets informatiques,michael.deruijter@dbf-autos.fr,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,...,NaN,NaN,NaN,NaN,NaN,NaN,Uncertain,Valid,Valid,Valid


In [70]:
#Email Duplicates Check (No email duplicates)
duplicates = dft[dft['Email'].duplicated()]
duplicates

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast


In [72]:
# LinkedIn / Address / Title / Email consistency Check
dft.describe()

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast
count,100,100,100,100,100,100,40,40,33,40,...,81,46,81,81,76,32,100,100,100,100
unique,100,100,79,98,97,100,31,28,19,29,...,43,12,49,6,46,32,6,2,2,2
top,eC_dhime_001,0033y00002OH1jhAAD,David,Lewis,CEO,aalvesy79@gmail.com,22 Adelaide Street W 41St Fl,Toronto,ON,M5H 4E3,...,Toronto,Ontario,M5H 4E3,Canada,(888) 432-4879,33 6 08 65 86 47,Yes,Valid,Valid,Valid
freq,1,1,6,2,2,1,4,6,6,6,...,11,11,11,28,11,1,79,99,99,99


In [59]:
# Auto Generate 5. Delivery tab 1 validated by filtering only Yes answers and Valid Matching and clean unwanted variables
dfd1 = dft[df['Still at the company?'] == 'Yes']
dfd1 = dfd1[['eC ID', 'Contact ID', 'First Name', 'Last Name', 'Title', 'Email',
       'Street Address', 'City', 'State/Province', 'Zip/Postal Code',
       'Country', 'Phone', 'Account ID', 'Account Name', 'Website',
       'LinkedIn URL', 'New Title', 'Job Level', 'Job Function', 'New Email',
       'Email Status', 'Mailing Street', 'Mailing City',
       'Mailing State/Province', 'Mailing Zip/Postal Code', 'Mailing Country',
       'Mobile Phone', 'Mobile Phone', 'Still at the company?']]
dfd1.head()

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing Street,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Mobile Phone,Mobile Phone,Still at the company?
0,eC_dhime_001,0033y00002OH1jhAAD,Adriano,Alves,Executivo de Novos Negócios e Inteligência de ...,aalvesy79@gmail.com,NaN,NaN,SP,NaN,...,"Rua Voluntários da Pátria, 3693, Santana",Sao Paulo,NaN,02401-300,Brazil,55 11 2972 8000,NaN,55 11 2972 8000,NaN,Yes
1,eC_dhime_003,0033y00002N5F0rAAF,Abraham,Vargas,Gerente de Operaciones,abraham.vargas@grabasa.mx,Cerrada Loma Bonita Número 204 Interior 33 Col...,Ciudad de Mexico,Ciudad de Mexico,11950,...,"Cerrada de Loma Bonita 33, Miguel Hidalgo",Mexico City,NaN,11950,Mexico,NaN,NaN,NaN,NaN,Yes
2,eC_dhime_004,0033y00002Q1TmMAAV,Auriane,Breton,Cheffe de Projets Marketing Opérationnel,abreton@mobalpa.com,NaN,NaN,NaN,NaN,...,18 Rue des Vernaies,Thones,NaN,74230,France,33 4 88 78 89 09,NaN,33 4 88 78 89 09,NaN,Yes
3,eC_dhime_005,0033y00002PnGyCAAV,Amaury,Decacqueray,Deputy Sales Director for Maritime Safety and ...,adecacqueray@ocea.fr,"Croisement, Avenue de Penhoet",Saint-Nazaire,Pays de la Loire,44600,...,Quai de la Cabaude,Les Sables d'Olonne,NaN,85100,France,33 2 51 21 05 90,33 6 08 65 86 47,33 2 51 21 05 90,33 6 08 65 86 47,Yes
4,eC_dhime_006,0033y00002NcyjsAAB,Adil,Husseini,Assistant Director Fundraising,adil.husseini@nationalemergenciestrust.org.uk,NaN,NaN,NaN,NaN,...,"London Scottish House, 95 Horseferry Road",London,England,SW1P 2DX,United Kingdom,NaN,NaN,NaN,NaN,Yes


In [60]:
# Auto Generate 5. Delivery tab 2 No longer with the company by filtering only No answers and clean the columns
dfd2 = dft[df['Still at the company?'] == 'No']
dfd2 = dfd2[['eC ID', 'Contact ID', 'First Name', 'Last Name', 'Title', 'Email',
       'Street Address', 'City', 'State/Province', 'Zip/Postal Code',
       'Country', 'Phone', 'Account ID', 'Account Name', 'Website',
       'LinkedIn URL','Still at the company?']]
dfd2.head(20)

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,Country,Phone,Account ID,Account Name,Website,LinkedIn URL,Still at the company?
31,eC_dhime_037,0033y00002NmHLuAAN,Arnaldo,Pereira Neto,Development Manager,arnaldopereiraneto2@gmail.com,NaN,NaN,MG,NaN,BR,+5511932897663,0013y00001c8LX7,Chronos São Carlos,https://www.chronos.org.br/,https://www.linkedin.com/in/arnaldo-p-neto,No
81,eC_dhime_092,0033y00002OB8vvAAD,David,Lós de Alcântara,Head of International Department,david.los@cbh.org.br,NaN,NaN,SP,NaN,BR,+55 21 98748-1094,0013y00001cZgf8,Confederação Brasileira de Hipismo - Brasil,http://www.cbh.org.br,https://www.linkedin.com/in/ddalc94,No
85,eC_dhime_121,0033y00002POZ4KAAX,Erwan,Green,Editor,egreen@atomiccartoons.com,NaN,NaN,NaN,NaN,CA,(310) 882-6265,0013y00001Y0Stm,Atomic Cartoons,https://thunderbird.tv,https://www.linkedin.com/in/ewan-green-6b2400b,No
88,eC_dhime_168,0033y00002NmHLQAA3,Gustavo,Ferracin,Sales Representative / Manager,gustavoferracin@usp.br,NaN,NaN,NaN,NaN,BR,NaN,0013y00001c8LX7,Chronos São Carlos,https://www.chronos.org.br/,https://www.linkedin.com/in/gustavoferracin,No
89,eC_dhime_216,0033y00002N00wjAAB,Juan,Nascimento,Coordenador de Parcerias e Novos Negócios,juan.nascimento@fas-amazonas.org,NaN,Manaus,NaN,NaN,BR,+55 11 94344-4513,0013y00001ce9Bu,Fundação Amazônia Sustentável,https://fas-amazonia.org/,https://www.linkedin.com/in/juanordn,No
90,eC_dhime_236,0033y00002OK62nAAD,Kenny,Singh,Senior Vice President- Client Relationship,kenny.singh@federalsoftsystems.com,NaN,NaN,NaN,NaN,US,NaN,0013y00001Y0T45,Federal Soft Systems Inc.,www.federalsoftsystems.com,https://www.linkedin.com/in/kenny-singh-ba0722245,No
94,eC_dhime_323,0033y00002N7eI6AAJ,Paul,Cannon,CEO,paul.s.cannon@gmail.com,22 Technology Parkway,Norcroft,GA,30092,US,4046643387,0013y00001Y0NS4,Catalyst Tech Ventures,catalysttechventures.com,https://www.linkedin.com/in/paulscannon,No
96,eC_dhime_372,0033y00002JaQdqAAF,sara,Dramé,Custumer Care Manager,sara@anybuddyapp.com,EURATECHNOLOGIES,Lille,FR-59,59000,FR,+33 6 19 60 94 73,0013y00001XzaY5,Anybuddy,www.anybuddyapp.com,https://www.linkedin.com/in/sara-dramé-b14867150,No
97,eC_dhime_386,0033y00002PNFcIAAX,Sébastien,Robert,Directeur du service de relations de travail,srobert@aptsq.com,NaN,NaN,NaN,NaN,CA,(450) 670-2411,0013y00001Y0Stp,Alliance du personnel professionnel et techniq...,www.aptsq.com,https://www.linkedin.com/in/sébastien-robert-9...,No
98,eC_dhime_402,0033y00002PQGxpAAH,Tristan,Goodes,3D Asset Supervisor,tgoodes@atomiccartoons.com,123 7th Ave W,Vancouver,BC,V5Y 1L8,CA,+1 604-734-2866,0013y00001Y0Stm,Atomic Cartoons,https://thunderbird.tv,https://www.linkedin.com/in/tristan-goodes,No


In [61]:
#Auto generate 5 Delivery tab 3 with no contacts by filtering Uncetain and clean unwanted variables
dfd3 = dft[df['Still at the company?'] == 'Uncertain']
dfd3 = dfd3[['eC ID', 'Contact ID', 'First Name', 'Last Name', 'Title', 'Email',
       'Street Address', 'City', 'State/Province', 'Zip/Postal Code',
       'Country', 'Phone', 'Account ID', 'Account Name', 'Website',
       'LinkedIn URL','Still at the company?']]
dfd3

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,Country,Phone,Account ID,Account Name,Website,LinkedIn URL,Still at the company?
86,eC_dhime_164,0033y00002O1fs9AAB,Gary,Smith,Fire Chief,gsmith@gomontrose.com,3960 Medina Rd,Fairlawn,OH,44333,US,(440) 934-6699,0013y00001Y0TDM,Ford Montrose Inc,www.montrosefordakron.com,NaN,Uncertain
87,eC_dhime_167,0033y00002OGyC6AAL,Gustavo,Lima Olmos Ferraz,Auxiliar de escritório,guolmos06@gmail.com,NaN,NaN,SP,NaN,BR,1123051773,0013y00001cjwHH,Hospital São Camilo Unidade Ipiranga,http://www.hospitalsaocamilosp.org.br/unidades...,NaN,Uncertain
91,eC_dhime_271,0033y00002OkX3eAAF,Madeline,Rutherford,Development Officer,madeline.rutherfoord@inspirenorth.co.uk,NaN,NaN,NaN,NaN,GB,7936947036,0013y00001ca1NA,Inspire North,www.inspirenorth.co.uk,NaN,Uncertain
93,eC_dhime_290,0033y00002NR8d4AAD,Michaël,Ruijter,Développeur et chargé de projets informatiques,michael.deruijter@dbf-autos.fr,NaN,NaN,NaN,NaN,FR,NaN,0013y00001Y0PL9,DBF AUTOMOBILES,www.dbf-autos.fr,NaN,Uncertain
95,eC_dhime_334,0033y00002NTZk2AAH,Carlos,Fernandes,Mr.,projetos@academiaperolasnegras.com.br,"Avenida Antão Bernardes, 3000, bairro de Forta...",Paty do Alferes,NaN,26950-000,BR,5521999532917,0013y00001cIq6g,Academia Pérolas Negras,http://academiaperolasnegras.org,NaN,Uncertain


In [62]:
# Check and generate Mismatch between Website Domain and Email one client found
dfm1 = dft[df['MatchingEmail'] == 'Invalid']
dfm1

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast
75,eC_dhime_086,0033y00002N5f0JAAR,DAMIEN,BOURSEREAU,Ingenieur commercial,damien.boursereau@bodet-software.com,BOULEVARD DU CORMIER,Cholet,FR-49,49300,...,Cholet,NaN,49302,France,33 825 814 400,33 7 84 24 60 52,Yes,Invalid,Mismatch between First/Last Name and Email,Mismatch between First/Last Name and Email


In [63]:
#Further check  Mismatched First/Last name with emal with the company by filtering only No answers same one client found
dfm2 = dft[df['MatchingFirst'] == 'Mismatch between First/Last Name and Email']
dfm3 = dft[df['MatchingLast'] == 'Mismatch between First/Last Name and Email']
dfm3

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,...,Mailing City,Mailing State/Province,Mailing Zip/Postal Code,Mailing Country,Mobile Phone,Mobile Phone,Still at the company?,MatchingEmail,MatchingFirst,MatchingLast
75,eC_dhime_086,0033y00002N5f0JAAR,DAMIEN,BOURSEREAU,Ingenieur commercial,damien.boursereau@bodet-software.com,BOULEVARD DU CORMIER,Cholet,FR-49,49300,...,Cholet,NaN,49302,France,33 825 814 400,33 7 84 24 60 52,Yes,Invalid,Mismatch between First/Last Name and Email,Mismatch between First/Last Name and Email


In [64]:
#Auto generate 5. Delivery 4 Mismatch between First/Last Name and Email dataset
dfm = dft[df['MatchingEmail'] == 'Invalid']
dfm = dfm[['eC ID', 'Contact ID', 'First Name', 'Last Name', 'Title', 'Email',
       'Street Address', 'City', 'State/Province', 'Zip/Postal Code',
       'Country', 'Phone', 'Account ID', 'Account Name', 'Website',
       'LinkedIn URL','Still at the company?']]
dfm

,eC ID,Contact ID,First Name,Last Name,Title,Email,Street Address,City,State/Province,Zip/Postal Code,Country,Phone,Account ID,Account Name,Website,LinkedIn URL,Still at the company?
75,eC_dhime_086,0033y00002N5f0JAAR,DAMIEN,BOURSEREAU,Ingenieur commercial,damien.boursereau@bodet-software.com,BOULEVARD DU CORMIER,Cholet,FR-49,49300,FR,825814400,0013y00001Y0PJP,Bodet Software,www.bodet-software.com,https://www.linkedin.com/in/damien-boursereau,Yes


In [65]:
#exporting excel sheets in one spreadsheet files for google sheets
with pd.ExcelWriter('mult_sheets_1.xlsx') as writer1:
    dfd1.to_excel(writer1, sheet_name = '5.Delivery Tab1 Validated', index = False)
    dfd2.to_excel(writer1, sheet_name = '5.Delivery Tab2 No Longer with', index = False)
    dfd3.to_excel(writer1, sheet_name = '5.Delivery Tab3 No Contacts', index = False)
    dfm.to_excel(writer1, sheet_name = '5.Delivery Tab4', index = False)